In [3]:
from flask import Flask, request, send_file
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
import pyperclip
import subprocess
import time
import json
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def efetuar_login(driver):
    # acessar icone do usuario
    try:
        icon_user = driver.find_element(By.XPATH, '//*[@id="hdr-c"]/nav/div/a[3]/i')
        icon_user.click()
        sleep(1)

        # acessar o login
        button_login = driver.find_element(By.XPATH, '//*[@id="user-menu"]/div[2]/div[1]/a[1]')
        button_login.click()
        sleep(3)

        # preencher login, senha e entrar
        input_user = driver.find_element(By.XPATH, '//*[@id="username"]')
        input_user.send_keys('mauriciopfleger')
        sleep(1)
        input_pass = driver.find_element(By.XPATH, '//*[@id="password"]')
        input_pass.send_keys('Jaqueta1!@')
        sleep(1)
        button_entrar = driver.find_element(By.XPATH, '//*[@id="allCnt"]/main/app-authentication/app-signin/div/form/button')
        button_entrar.click()
        sleep(5)
        print('realizado login')
    except:
        print('logado')

def apostar(driver, valor_aposta, jogo_unico):
    apostas = driver.find_element(By.XPATH, '//*[@id="quick-bet"]/div[2]/div[1]/div[2]/i')
    apostas.click()
    sleep(1)
    
    if jogo_unico:                                  
        valor = driver.find_element(By.XPATH, '//*[@id="single-betslip"]/div[1]/app-coupon/div/div[2]/div[2]')
    else:                                               
        valor = driver.find_element(By.XPATH, '//*[@id="multi-betslip"]/div[2]/div[2]/div[1]/div/div/div[2]')
    valor.click()  
    sleep(1)

    for num in valor_aposta:
        vnum = num
        if num == '0':
            vnum = '10'
        valor_num = driver.find_element(By.XPATH, f'//*[@id="cNumpad"]/app-custom-numpad/div[3]/button[{vnum}]')
        valor_num.click()
        sleep(1)
    
    enter = driver.find_element(By.XPATH, '//*[@id="cNumpad"]/app-custom-numpad/div[3]/button[13]')
    enter.click()
    sleep(1)
    try:
        if jogo_unico:
            aceitar = driver.find_element(By.XPATH, '//*[@id="single-betslip"]/div[2]/div[2]/div[2]/a')
        else:     
            aceitar = driver.find_element(By.XPATH, '//*[@id="multi-betslip"]/div[2]/div[2]/div[2]/a')
        if (aceitar.text.strip() == 'Aceitar todas as alterações'):
            aceitar.click()
            sleep(1)
    except:
        print('direto')
                                                                
    apostar = driver.find_element(By.XPATH, '//*[@id="betslip-placebet-btn"]')
    apostar.click()
    sleep(1)
    voltar = driver.find_element(By.XPATH, '//*[@id="preview-betslip"]/div[3]/a[1]')
    voltar.click()
    sleep(1)

erro = False

def caputrar_esportedasorte():
    # Especifica o caminho do arquivo que deseja excluir
    caminho_arquivo = "partidas.csv"

    # Verifica se o arquivo existe antes de excluí-lo
    if os.path.exists(caminho_arquivo):
        os.remove(caminho_arquivo)
        print(f"O arquivo {caminho_arquivo} foi excluído com sucesso")
    else:
        print(f"O arquivo {caminho_arquivo} não existe")

    chrome_options = Options()                   
    chrome_options.add_argument("--user-data-dir=C:\\Users\\mauri\\AppData\\Local\\Google\\Chrome\\User Data\\Default") 
    driver = webdriver.Chrome(options=chrome_options)    

    # acessar o site
    driver.get("https://m.esportesdasorte.com/ptb")
    sleep(8)

    erro = False    
    try:
        efetuar_login(driver)

        lista_campeonato = ['brasileirao'] #, 'libertadores'

        registros_resultado = []
        registros_dupla_chance = []

        for campeonato in lista_campeonato:
            try:
                if (campeonato == 'brasileirao'):
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/brazil")
                elif (campeonato == 'libertadores'):                    
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/international-clubs")
                    
                sleep(8)

                for i in range(2):
                    if i == 1: 
                        if (campeonato == 'brasileirao'):
                            dupla_chance = driver.find_element(By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list/ul/li[5]/a')

                        elif (campeonato == 'libertadores'):
                            dupla_chance = driver.find_element(By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list[1]/ul/li[5]/a')

                        dupla_chance.click()
                        sleep(1)                    
                    
                    divs_resultado = driver.find_elements(by=By.CLASS_NAME, value='match-content')
                    for div in divs_resultado:
                        # div.text = 'Vitória BA \n Goianiense \n 18/05 16:00 \n BB \n +354 \n equalizer \n 1.94 \n 3.34 \n 4.45'
                        infos = div.text.split('\n')
                        time_casa = infos[0]
                        time_visitante = infos[1]
                        data = infos[2]

                        odd_casa = ""
                        odd_empate = ""
                        odd_fora = ""
                        odd_1x = ""
                        odd_12 = ""
                        odd_x2 = ""

                        if i == 0:
                            odd_casa = infos[len(infos)-3]
                            odd_empate = infos[len(infos)-2]
                            odd_fora = infos[len(infos)-1]
                            registros_resultado.append([campeonato, time_casa, time_visitante, data, odd_casa, odd_empate, odd_fora])
                        else: 
                            odd_1x = infos[len(infos)-3]
                            odd_12 = infos[len(infos)-2]
                            odd_x2 = infos[len(infos)-1]
                            registros_dupla_chance.append([campeonato, time_casa, time_visitante, data, odd_1x, odd_12, odd_x2])
            except:
                continue

        driver.quit() 
                
        df_resultado = pd.DataFrame(registros_resultado, columns=['campeonato', 'time_casa', 'time_visitante', 'data', 'odd_casa', 'odd_empate', 'odd_visitante'])
        df_dupla_chance = pd.DataFrame(registros_dupla_chance, columns=['campeonato', 'time_casa', 'time_visitante', 'data', 'odd_1x', 'odd_12', 'odd_x2'])
        new_df = pd.concat([df_resultado, df_dupla_chance[['odd_1x', 'odd_12', 'odd_x2']]], axis=1)
        new_df.to_csv(caminho_arquivo, index=False, sep=';', encoding='utf-8')

        print('capturado')
    except: 
        erro = True
        driver.quit()
    
    return erro

def apostar_esportedasorte():
    file = 'bilhetes.csv'
    df = pd.read_csv(file, delimiter=';')
    df.set_index('Numero', inplace=True)

    chrome_options = Options()                   
    chrome_options.add_argument("--user-data-dir=C:\\Users\\mauri\\AppData\\Local\\Google\\Chrome\\User Data\\Default") 
    driver = webdriver.Chrome(options=chrome_options)    

    # acessar o site
    driver.get("https://m.esportesdasorte.com/ptb")
    sleep(8)

    erro = False
    try:
        efetuar_login(driver)

        try:                                                  
            apostas = driver.find_element(By.XPATH, '//*[@id="quick-bet"]/div[2]/div[1]/div[2]/i')
            apostas.click()
            sleep(1)
            try:
                apagar = driver.find_element(By.XPATH, '//*[@id="single-betslip"]/div[2]/div[2]/div[2]/a[1]')
            except:
                apagar = driver.find_element(By.XPATH, '//*[@id="multi-betslip"]/div[2]/div[2]/div[3]/a[1]')
            apagar.click()
            sleep(1)
        except:
            print('sem apostas')

        previous_campeonato = None
        previous_index = 1
        valor_aposta = '0'
        qtd_jogos = 0
        skipou = False
        for index, row in df.iterrows():
            if row['Apostado'] == 1:
                previous_index = index
                skipou = True
                continue

            qtd_jogos += 1
            if (index != previous_index) and (not skipou):
                apostar(driver, valor_aposta, qtd_jogos == 1)
                qtd_jogos = 0

            previous_index = index
            
            skipou = False
            
            valor_aposta = str(row['Aposta'])
            print('valor_aposta', valor_aposta)
            
            if (row['Campeonato'] == 'brasileirao'):
                # acessar o site
                if previous_campeonato != row['Campeonato']:
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/brazil")
                    sleep(4)
                else: 
                    resultado = driver.find_element(By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list/ul/li[1]/a')
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", resultado)
                    driver.execute_script("window.scrollBy(0, -100);")
                    resultado.click()
            elif (row['Campeonato'] == 'libertadores'):
                # acessar o site
                if previous_campeonato != row['Campeonato']:
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/international-clubs")
                    sleep(4)
                else: 
                    resultado = driver.find_element(By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list[1]/ul/li[1]/a')
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", resultado)
                    driver.execute_script("window.scrollBy(0, -100);")
                    resultado.click()
            
            sleep(1)

            print('passou do campeonato')
            
            previous_campeonato = row['Campeonato']

            for i in range(2):
                if i == 1: 
                    if (row['Campeonato'] == 'brasileirao'):
                        dupla_chance = driver.find_element(By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list/ul/li[5]/a')

                    elif (row['Campeonato'] == 'libertadores'):
                        dupla_chance = driver.find_element(By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list[1]/ul/li[5]/a')

                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", dupla_chance)                 
                    dupla_chance.click()
                    sleep(1)

                parar = False
                divs_resultado = driver.find_elements(by=By.CLASS_NAME, value='match-content')
                for div in divs_resultado:
                    # div.text = 'Vitória BA \n Goianiense \n 18/05 16:00 \n BB \n +354 \n equalizer \n 1.94 \n 3.34 \n 4.45'
                    infos = div.text.split('\n')
                    time_casa = infos[0]
                    time_visitante = infos[1]
                    data = infos[2]

                    jogo = str.split(row['Jogo'], ' x ')
                    dftime_casa = jogo[0]
                    dftime_visitante = jogo[1]
                    dfdata = row['Data']

                    if (time_casa == dftime_casa and
                        time_visitante == dftime_visitante and
                        data == dfdata):
                        elementos_internos = div.find_elements(By.TAG_NAME, 'a')  
                        if (i == 0 and pd.isna(row['Resultado2'])):
                            if (row['Resultado1'] == time_casa):                            
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[1])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[1].click()
                                sleep(1)
                                parar = True
                                break
                            elif (row['Resultado1'] == time_visitante): 
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[3])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[3].click()
                                sleep(1)
                                parar = True
                                break
                            elif (row['Resultado1'] == 'Empate'):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[2])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[2].click()
                                sleep(1)
                                parar = True
                                break
                        elif (i == 1 and not pd.isna(row['Resultado2'])):
                            if (row['Resultado1'] == time_casa and row['Resultado2'] == 'Empate'):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[1])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[1].click()
                                sleep(1)
                                break
                            elif (row['Resultado1'] == 'Empate' and row['Resultado2'] == time_visitante):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[3]) 
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[3].click()
                                sleep(1)
                                break
                            elif (row['Resultado1'] == time_casa and row['Resultado2'] == time_visitante):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[2])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[2].click()
                                sleep(1)
                                break

                if parar:
                    break

        apostar(driver, valor_aposta, qtd_jogos == 1)

        driver.quit() 

        print('apostado')
    except:
        erro = True
        driver.quit() 
    
    return erro

def credito_esportedasorte(valor):
    chrome_options = Options()                   
    chrome_options.add_argument("--user-data-dir=C:\\Users\\mauri\\AppData\\Local\\Google\\Chrome\\User Data\\Default") 
    driver = webdriver.Chrome(options=chrome_options)
    
    # acessar o site
    driver.get("https://m.esportesdasorte.com/ptb")
    sleep(8)

    erro = False     
    try: 
        efetuar_login(driver)

        icon_user = driver.find_element(By.XPATH, '//*[@id="hdr-c"]/nav/div/span/a/i')
        icon_user.click()
        sleep(1)

        depositar = driver.find_element(By.XPATH, '//*[@id="user-menu"]/div[2]/ul/li[1]/ul/li/div/ul/li[1]/a')
        depositar.click()
        sleep(1)
        
        sleep(1)
        driver.execute_script("window.scrollBy(0, 600);")
        sleep(1)

        paybrokers = driver.find_element(By.XPATH, '//*[@id="allCnt"]/main/app-dashboard/payment-component/deposit-money/deposit-withdraw-money/div[2]/div/div/ul/li[1]/a')
        paybrokers.click()
        sleep(1)

        input = driver.find_element(By.XPATH, '//*[@id="mntNpt"]')
        input.send_keys(valor)
        sleep(1)

        finalizar = driver.find_element(By.XPATH, '//*[@id="allCnt"]/main/app-dashboard/payment-component/payment-form-out/payment-form/div[2]/div/div/form/div/div/button')
        finalizar.click()
        sleep(5)
                                                         
        copiar = driver.find_element(By.XPATH, '//*[@id="allCnt"]/main/app-dashboard/payment-component/payment-result/div/div/div/div[2]/div[4]/div/button')
        copiar.click()
        sleep(1)
        
        qrcode = pyperclip.paste()
        print(qrcode)
        
        driver.quit()
        return qrcode        
    except:
        erro = True
        driver.quit()

    return erro

def saldo_esportedasorte():
    chrome_options = Options()                   
    chrome_options.add_argument("--user-data-dir=C:\\Users\\mauri\\AppData\\Local\\Google\\Chrome\\User Data\\Default") 
    driver = webdriver.Chrome(options=chrome_options)
    
    # acessar o site
    driver.get("https://m.esportesdasorte.com/ptb")
    sleep(8)

    erro = False     
    try: 
        efetuar_login(driver)

        icon_user = driver.find_element(By.XPATH, '//*[@id="hdr-c"]/nav/div/span/a/i')
        icon_user.click()
        sleep(1)

        texto_valor = driver.find_element(By.XPATH, '//*[@id="user-menu"]/div[2]/div/div[4]/a[2]/span[2]')
        valor = texto_valor.text.replace('R$ ', '')        

        driver.quit()

        return valor
    except:
        erro = True
        driver.quit()
    
    return erro

def read_output(output_queue, process):
    for line in iter(process.stdout.readline, ''):
        output_queue.put(line)
        output_queue.put(None)

def start_ngrok():
    # Caminho completo para o executável do ngrok
    process = subprocess.Popen(["C:\\Projetos\\ngrok-v3-stable-windows-amd64\\ngrok.exe", "http", "5032"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    # Aguardando um tempo para o ngrok se iniciar e gerar o endereço
    time.sleep(5)

    # Recupere o status do ngrok
    status = subprocess.check_output(['curl', '-s', 'http://localhost:4040/api/tunnels'])
    tunnels_info = json.loads(status)

    #process.terminate()

    # Extraia e retorne a URL
    public_url = tunnels_info['tunnels'][0]['public_url']
    return public_url

def send_email(subject, body, to_email):
    # Configurações do servidor SMTP e da conta de e-mail
    smtp_server = 'smtp.gmail.com'
    smtp_port = 465
    from_email = 'mauriciopfleger7@gmail.com'
    from_password = 'mfht xlpv iwpt zalr'

    # Criação do objeto de mensagem
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    # Adicionar o corpo do e-mail
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Conectar ao servidor SMTP
        server = smtplib.SMTP_SSL(smtp_server, smtp_port)
        #server.starttls()  # Criptografar a conexão
        server.login(from_email, from_password)
        server.sendmail(from_email, to_email, msg.as_string())
        print('E-mail enviado com sucesso!')
    except Exception as e:
        print(f'Erro ao enviar e-mail: {e}')
    finally:
        server.quit()
    
    
# credito_esportedasorte(10)

app = Flask(__name__)

@app.route('/buscar')
def run_buscar():

    erro = caputrar_esportedasorte()
    
    caminho_arquivo = "partidas.csv"

    if erro:
        return "Ocorreu um erro", 400

    return send_file(
        caminho_arquivo,
        mimetype='text/csv',
        as_attachment=True,
        download_name=caminho_arquivo
    )

@app.route('/apostar', methods=['POST'])
def run_apostar():
    if 'file' not in request.files:
        return "No file part in the request", 400
    file = request.files['file']
    if file.filename == '':
        return "No file selected", 400
    
    #Especifica o caminho do arquivo que deseja excluir
    caminho_arquivo = "bilhetes.csv"

    #Verifica se o arquivo existe antes de excluí-lo
    if os.path.exists(caminho_arquivo):
        os.remove(caminho_arquivo)
        print(f"O arquivo {caminho_arquivo} foi excluído com sucesso")
    else:
        print(f"O arquivo {caminho_arquivo} não existe")

    df = pd.read_csv(file, delimiter=';')
    df.to_csv(caminho_arquivo, sep=';', index=False)  # Salva o DataFrame sem a coluna de índice

    erro = apostar_esportedasorte()

    if erro:
        return "Ocorreu um erro", 400

    return "Apostas realizadas!"

@app.route('/credito')
def run_credito():
    if 'valor' not in request.args:
        return "No 'valor' in the request", 400
    valor = request.args['valor']   

    result = credito_esportedasorte(valor)

    if result is bool and result:
        return "Ocorreu um erro", 400
    
    return result

@app.route('/saldo')
def run_saldo():
    result = saldo_esportedasorte()

    if result is bool and result:
        return "Ocorreu um erro", 400
    
    return result

@app.route('/server')
def run_server():
    result = start_ngrok()
    
    return result

if __name__ == '__main__':
    url = start_ngrok()
    send_email('Servidor', url, 'mauriciopfleger7@gmail.com')
    app.run(host='0.0.0.0', port=5032, debug=False)    

E-mail enviado com sucesso!
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5032
 * Running on http://192.168.1.2:5032
Press CTRL+C to quit


O arquivo bilhetes.csv foi excluído com sucesso
logado
sem apostas
valor_aposta 10
passou do campeonato
valor_aposta 10
passou do campeonato
valor_aposta 10
passou do campeonato
apostado


127.0.0.1 - - [09/Aug/2024 23:49:24] "POST /apostar HTTP/1.1" 200 -


logado


127.0.0.1 - - [09/Aug/2024 23:49:45] "GET /saldo HTTP/1.1" 200 -
